# Imports

In [ ]:
import os
import json
import re
from pathlib import Path
import pandas as pd
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
def create_prompt(row):
  # SYSTEM = "Você é um assistente especializado em analisar e identificar notícias falsas ou verdadeiras."
  # <|start_header_id|>system<|end_header_id|>{SYSTEM}<|eot_id|>
  INPUT = f'''Dada a notícia abaixo, classifique-a como "falsa" ou "verdadeira": {row["preprocessed_news"]}'''
  OUTPUT = row["label"]

  return f'''<|begin_of_text|><|start_header_id|>user<|end_header_id|>{INPUT}<|eot_id|>
  <|start_header_id|>assistant<|end_header_id|>{OUTPUT}<|eot_id|>'''

# Base line websites

## English

In [ ]:
realNews_path = os.path.join(DATASET_DIR, "realNewsWebsites.json")
with open(realNews_path) as file:
    real_website_urls = json.load(file)

real_website_urls

## Brazil

In [ ]:
'''
# The script below was used to create a csv from txt files, since it's to expansive to upload all txt files in google drive. Need to download true-meta-infomation folder
# from Fakebr corpus and run locally this script
import pandas as pd
def convert_meta_txt_to_csv(folder, output_file_name):
    columns = "author,link,category,date of publication,number of tokens,number of words without punctuation,number of types,number of links inside the news,number of words in upper case,number of verbs,number of subjuntive and imperative verbs,number of nouns,number of adjectives,number of adverbs,number of modal verbs (mainly auxiliary verbs),number of singular first and second personal pronouns,number of plural first personal pronouns,number of pronouns,pausality,number of characters,average sentence length,average word length,percentage of news with speeling errors,emotiveness,diversity"
    columns = columns.split(",")
    all_txt_files = os.listdir(folder)
    df = pd.DataFrame(columns=columns)
    for txt_name in all_txt_files:
        txt_file = f"{true_meta_data}/{txt_name}"
        with open(txt_file, encoding="utf8") as file:
            lines = file.readlines()

        new_row = dict(zip(columns,lines))
        df.loc[len(df)] = new_row

    file_name = f"{os.getcwd()}/{output_file_name}.csv"
    df.to_csv(file_name, index=False)

folder = f"{DATASET_DIR}/Fakebr/full_texts/true-meta-information"
output_file_name = "true_meta_data"
convert_meta_txt_to_csv(folder, output_file_name)
'''

import pandas as pd

custom_br_websites = '''g1.globo.com
folha.uol.com.br
oglobo.globo.com
estadao.com.br
noticias.uol.com.br
terra.com.br
r7.com
veja.abril.com.br
epoca.globo.com
valor.globo.com
correiobraziliense.com.br
jb.com.br
g1.globo.com/jornal-nacional
opovo.com.br
gazetadopovo.com.br
extra.globo.com
cartacapital.com.br
brasil247.com
redebrasilatual.com.br
em.com.br
agazeta.com.br
jovempan.com.br
band.uol.com.br/noticias
exame.com
folhape.com.br
bbc.com/portuguese
cnnbrasil.com.br
metropoles.com
istoe.com.br
revistaforum.com.br
jornaldebrasilia.com.br
gazetaonline.com.br
noticiasaominuto.com.br
ultimosegundo.ig.com.br
jornaldocomercio.com
brasildefato.com.br
poder360.com.br
congressoemfoco.uol.com.br
nexojornal.com.br
gazetaesportiva.com
placar.abril.com.br
lance.com.br
infomoney.com.br
agazetadoacre.com
www.gazetape.com
folhadelondrina.com.br
oprogressonet.com
jornaldotocantins.com.br
folhabv.com.br
acritica.com'''.split('\n')

def get_all_domains(file):

    csv = pd.read_csv(file)
    links = list(csv["link"])

    all_domains = set()
    for link in links:
        domain = link.split("/")[2]
        all_domains.add(domain)

    return all_domains

true_meta_data = f"{DATASET_DIR}/Fakebr/true_meta_data.csv"
domains = get_all_domains(true_meta_data)
for new_domain in custom_br_websites:
    domains.add(new_domain)

print(domains, len(domains))
domains_txt_file =  f"{DATASET_DIR}/realbrsites.txt"
with open(domains_txt_file, "w") as file:
  for domain in domains:
    file.write(f"{domain}\n")

{'noticiasaominuto.com.br', 'gazetaesportiva.com', 'folhabv.com.br', 'acervo.estadao.com.br', 'g1.globo.com/jornal-nacional', 'emais.estadao.com.br', 'jovempan.com.br', 'veja.abril.com.br', 'exame.com', 'jornaldocomercio.com', 'g1.globo.com', 'lance.com.br', 'infomoney.com.br', 'brasil.estadao.com.br', 'cnnbrasil.com.br', 'ultimosegundo.ig.com.br', 'politica.estadao.com.br', 'brasildefato.com.br', 'esportes.estadao.com.br', 'gazetadopovo.com.br', 'extra.globo.com', 'r7.com', 'link.estadao.com.br', 'oprogressonet.com', 'epoca.globo.com', 'correiobraziliense.com.br', 'jornaldotocantins.com.br', 'alias.estadao.com.br', 'jornaldebrasilia.com.br', 'www.estadao.com.br', 'agazetadoacre.com', 'sustentabilidade.estadao.com.br', 'terra.com.br', 'datafolha.folha.uol.com.br', 'saude.estadao.com.br', 'brasil247.com', 'www.territorioeldorado.limao.com.br', 'redebrasilatual.com.br', 'cultura.estadao.com.br', 'agazeta.com.br', 'band.uol.com.br/noticias', 'f5.folha.uol.com.br', 'noticias.uol.com.br', '

# Brasil Portuguese Dataset (Fake.br-Corpus)

## full text version

In [ ]:
'''
# The script below was used to create a csv from txt files, since it's to expansive to upload all txt files in google drive. Need to download true-meta-infomation folder
# from Fakebr corpus and run locally this script
import pandas as pd
def convert_txt_to_df(folder, label):
    columns = ["label", "text"]
    all_txt_files = os.listdir(folder)
    df = pd.DataFrame(columns=columns)
    for txt_name in all_txt_files:
        txt_file = f"{true_meta_data}/{txt_name}"
        with open(txt_file, encoding="utf8") as file:
            lines = file.readlines()

        truncated_str = "".join([string.strip("\n") for string in lines])
        new_row = [label, truncated_str]
        df.loc[len(df)] = new_row

    return df

true_folder = f"{DATASET_DIR}/Fakebr/full_texts/true"
fake_folder = f"{DATASET_DIR}/Fakebr/full_texts/fake"
true_df = convert_txt_to_df(true_folder, label=1)
fake_df = convert_txt_to_df(fake_folder, label=0)

df = pd.concat([true_df, fake_df])
df.to_csv(f"{DATASET_DIR}/Fakebr/full_text_fakebrcorpus.csv",index=False)
'''

In [ ]:
fakebrcorpus_full = pd.read_csv(f"{DATASET_DIR}/Fakebr/full_text_fakebrcorpus.csv")
fakebrcorpus_full

# LIAR Dataset ("Liar, Liar Pants on Fire": A New Benchmark Dataset for Fake News Detection)

In [ ]:
liar_train = pd.read_csv(f'{DATASET_DIR}/LIAR/train.tsv', sep='\t', header=None)
liar_test = pd.read_csv(f'{DATASET_DIR}/LIAR/test.tsv', sep='\t', header=None)
liar_valid = pd.read_csv(f'{DATASET_DIR}/LIAR/valid.tsv', sep='\t', header=None)

# liar_train.columns = ["label", "text"]
# liar_test.columns = ["label", "text"]
# liar_valid.columns = ["label", "text"]

LIAR = pd.concat([liar_train, liar_test, liar_valid])
LIAR = LIAR[LIAR[1].isin(['true','false'])]
LIAR

In [ ]:
LIAR[LIAR[1]=='true']

In [ ]:
liar = pd.read_csv(f'{DATASET_DIR}/LIAR/train.tsv', sep='\t', header=None)
liar

In [ ]:
liar_with_columns = liar[[1, 2]]
liar_with_columns.columns = ["label", "text"]
liar_with_columns

In [ ]:
liar_train_v2 = liar_with_columns[(liar_with_columns["label"] == "true") | (liar_with_columns["label"] == "false")]
liar_train_v2.to_csv(f'{DATASET_DIR}/LIAR/liar_train_v2.csv',
                      header=["label", "text"], index=False)

In [ ]:
liar_train_v2[liar_train_v2['label']=='false']

In [ ]:
liar_test = pd.read_csv(f'{DATASET_DIR}/LIAR/test.tsv', sep='\t', header=None)
liar_test_with_columns = liar_test[[1, 2]]
liar_test_with_columns.columns = ["label", "text"]
liar_test_v2 = liar_test_with_columns[(liar_test_with_columns["label"] == "true") | (liar_test_with_columns["label"] == "false")]
liar_test_v2.to_csv(f'{DATASET_DIR}/LIAR/liar_test_v2.csv',
                      header=["label", "text"], index=False)
liar_test_v2

## final version

In [ ]:
liar_full = pd.read_csv(f'{DATASET_DIR}/LIAR/liar_train.csv')
liar_full["label"] = liar_full["label"].replace({False: 0, True: 1})
liar_full

In [ ]:
liar_full.to_csv(f'{DATASET_DIR}/LIAR/liar_full.csv', index=False)

In [ ]:
liar_full = pd.read_csv(f'{DATASET_DIR}/LIAR/liar_full.csv')
liar_full

In [ ]:
len(liar_full[liar_full['label'] == 0]), len(liar_full[liar_full['label'] == 1])

(3990, 3352)

In [ ]:
len(liar_full[liar_full['label'] == 0])